In [1]:
from setproctitle import setproctitle
setproctitle("Wav2Vec2 textfile")

import os

from tqdm import tqdm

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
kss_path = os.path.join(dataset_path, "kss")
zeroth_path = os.path.join(dataset_path, "zeroth")
train_zeroth_path = os.path.join(zeroth_path, "train_data")
test_zeroth_path = os.path.join(zeroth_path, "test_data")
klec_path = os.path.join(dataset_path, "klec_12h_wav")
kspon_path = os.path.join(dataset_path, "KsponSpeech")

In [2]:
import os
import torch

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

# _net = ResNet50().cuda()
# net = nn.DataParallel(_net).to(device)

Device: cuda
Count of using GPUs: 2
Current cuda device: 0


# 음성파일에 맞는 텍스트 파일 생성

In [30]:
'''
음성파일 개수에 맞는 텍스트 파일 생성

1. kss 전사 텍스트 파일 생성
2. zeroth 전사 텍스트 파일 생성
'''

## ver 1. kss
with open(os.path.join(dataset_path, "transcript.v.1.4.txt"), 'r') as f:  # 전체적인 전사 내용이 담긴 파일 읽기
    data = f.read()
    
data = data.split("\n")

for text_data in tqdm(data):
    file_data = text_data.split("|")
    wav_path = os.path.join(kss_path, file_data[0])
    txt_path = wav_path.replace(".wav", ".txt")
    with open(txt_path, 'w') as f:
        f.write(file_data[2])  # file_data[1]은 숫자 123을 '백이심삽'이 아닌 숫자 그대로 123으로 표현했기에 발음 그대로 적혀있는 file_data[2]를 적용

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12854/12854 [00:01<00:00, 10424.18it/s]


In [16]:
# a = '''106_003_1720 약 백 명의 성경 연구생이 포즈디쇼브체 마을에서 모였습니다
# 106_003_1749 홍어는 일베 사용자들이 전라도민들을 비하할 때 쓰는 단어이다'''
# a = a.split('\n')
# print(a[0][:12])
# print(a[0][:12] + '.flac')
# print(a[0][13:])

106_003_1720
106_003_1720.flac
약 백 명의 성경 연구생이 포즈디쇼브체 마을에서 모였습니다


In [4]:
## ver 2. zeroth
# for folder in tqdm(sorted(os.listdir(train_zeroth_path))):
for folder in tqdm(sorted(os.listdir(test_zeroth_path))):
    wav_list = list()
    text_list = list()
    # folder_path = os.path.join(train_zeroth_path, folder)
    folder_path = os.path.join(test_zeroth_path, folder)

    text_file = os.path.join(folder_path, f'{folder}_003.trans.txt')
    with open(text_file, 'r') as f:
        text_data = f.read()

    text_data = text_data.split('\n')
    for data in text_data:
        wav_list.append(data[:12] + ".flac")
        text_list.append(data[13:])
    
    for wav_file, text in zip(wav_list, text_list):
        text_data_file = os.path.join(folder_path, wav_file.replace(".flac", ".txt"))
        with open(text_data_file, 'w') as w:
            w.write(text)
    
    os.remove(text_file)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 272.05it/s]


In [5]:
print(wav_list[:3])
print(text_list[:3])

['149_003_0057.flac', '149_003_0164.flac', '149_003_0200.flac']
['이 일 자동차업계에 따르면 임팔라는 지난달 삼십 일 일 하루에만 구백 대가 넘는 계약이 이뤄진 것으로 파악됐다', '우리는 위기의 진앙이 아닌 안전지대가 됐다는 말까지 나올 정도였다', '성빈센트병원 제공신경외과 전문의들은 흔히 혈관을 택배기사에 비유한다']


# Vocab 생성

In [3]:
from setproctitle import setproctitle
setproctitle("Wav2Vec2 Vocab")

from tqdm import tqdm
from datasets import Dataset, ClassLabel
from IPython.display import display, HTML
from glob import glob
from kspon_preprocess import special_filter, bracket_filter

import re
import librosa
import random
import numpy as np
import pandas as pd
import os


def show_random_elements(dataset, num_examples=15):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    
    df = pd.DataFrame(dataset[:num_examples])
    display(HTML(df.to_html()))

    
def _read_txt_file(file_path):
    # with open(file_path, 'r', encoding='utf-8') as f:
    with open(file_path, 'r', encoding='cp949') as f:
        text = f.read()
        
    return text


text_list = list()

## ver1. kss
# for folder in tqdm(os.listdir(kss_path)):
#     folder_path = os.path.join(kss_path, folder)
#     for file in tqdm(sorted(os.listdir(folder_path))):
#         if file.endswith(".txt"):
#             text_file = os.path.join(folder_path, file)
#             with open(text_file, 'r') as f:
#                 text_data = f.read()
        
#             text_list.append(text_data)  # 총 12854

## ver2. zeroth
# for folder in tqdm(sorted(os.listdir(train_zeroth_path))):
#     folder_path = os.path.join(train_zeroth_path, folder)
#     for file in sorted(os.listdir(folder_path)):
#         if file.endswith(".txt"):
#             text_file = os.path.join(folder_path, file)
#             with open(text_file, 'r') as f:
#                 text_data = f.read()
        
#             text_list.append(text_data)  # 총 22368

## ver3. klecspeech
# for file in tqdm(sorted(os.listdir(klec_path))):
#     if file.endswith(".txt"):
#         text_file = os.path.join(klec_path, file)
#         text = _read_txt_file(text_file)
#         text_list.append(text)

## ver4. ksponspeech
audio_list = list()
durations = 0
max_sec = 10.0
min_sec = 2.0

kspon_wavs = glob(os.path.join(kspon_path, '**', '*.wav'), recursive=True)
random.seed(44)
random.shuffle(kspon_wavs)

remove_re = '[a-zA-Z0-9%]'

for file in tqdm(kspon_wavs):
    duration = librosa.get_duration(filename=file, sr=16000)
    if (min_sec <= duration) and (max_sec >= duration):
        text_path = file.replace(".wav", ".txt")
        text = _read_txt_file(text_path)
        text = special_filter(bracket_filter(text))
        if re.findall(remove_re, text) == []:
            text_list.append(text)
            audio_list.append(file)
            durations += duration
    if durations >= 500*60*60:
        break

    # 대략 50~60% 근처에서 tqdm 정지


len(text_list)

 76%|████████████████████████████████████████████████████████████████████████████████████▎                          | 464890/612422 [03:34<01:07, 2172.30it/s]


KeyboardInterrupt: 

In [ ]:
text_dict = {"text": text_list}

vocab_timit = Dataset.from_dict(text_dict)
print(len(vocab_timit))
print(vocab_timit)

show_random_elements(vocab_timit)

In [ ]:
print(vocab_timit['text'][:5])
print(vocab_timit['text'][-5:])

In [6]:
from kspon_preprocess import special_filter, bracket_filter
import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    # batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"])
    batch["text"] = special_filter(bracket_filter(batch["text"]))
    
    return batch

vocab_timit = vocab_timit.map(remove_special_characters)
show_random_elements(vocab_timit)

  0%|          | 0/235442 [00:00<?, ?ex/s]

,text
0,너무 막 완결 된 거 막 추천하는 거 없냐 요즘
1,음 어쨌든 한번 사 봐 이 인생이 편해져
2,난 너가 언제 한번 대마초 하다가 걸려갖고 잡혀갔음 좋겠어
3,그 그 그런 이 그런 이유
4,아침 많이 먹긴 하는데
5,그래서 막
6,햄 치즈 응 맛있던데 그 빵이 빠삭빠삭해서
7,그러니까
8,무슨 아쿠아 뭐였거든 아쿠아맨인가
9,나는 걔가 우리 애들 중에 제일 애 같은 거 같애


In [ ]:
# def extract_all_chars(batch):
#     all_text = " ".join(batch["text"])
#     vocab = list(set(all_text))
    
#     return {"vocab": vocab, "all_text": [all_text]}

# vocabs = vocab_timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True)

def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = extract_all_chars(vocab_timit)

In [ ]:
vocab_list = list(set(vocabs["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

In [ ]:
import json
import os

vocab_path = os.path.join(dataset_path, 'vocab.json')

In [ ]:
with open(vocab_path, 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Processor 생성

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(vocab_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

In [ ]:
# tokenizer.save_pretrained(dataset_path)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
feature_extractor

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
# processor = Wav2Vec2Processor.from_pretrained(dataset_path)
processor

In [44]:
processor.save_pretrained(dataset_path)

# 학습 및 테스트 데이터셋 구축

In [ ]:
from setproctitle import setproctitle
setproctitle("Wav2Vec2 Dataset")

from datasets import Dataset
from transformers import Wav2Vec2Processor
from kspon_preprocess import special_filter, bracket_filter, del_noise  # 특수 기호 제거하는 전처리 코드 함수
from tqdm import tqdm

import numpy as np
import os
import soundfile as sf
import librosa
import re


'''
timit : Transformers가 제input_length_list 데이터셋 구조를 맞추기 위해 사용한 데이터셋 변수로,

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

와 같은 데이터 구조를 지니고 있다.

이때 timit['train'][0]을 입력하여 세부 내용을 보면,

{'audio': {'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
         -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
  'sampling_rate': 16000},
 'file': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
 'text': 'Would such an act of refusal be useful?'}
 
으로, stt 솔루션에서는 바로 학습데이터셋 양식에 맞추기 위해 input_values, input_lengths, labels만 입력받을 예정이다.

Wav2Vec2는 16kHz(sample_rate=16000)의 '1차원' 배열 형식으로 입력을 예상하므로, 오디오 파일을 로드하고 다시 리샘플링해야한다.
'''

# processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")  # hugging face에서 제공된 한국어 고성능 pretrained 모델의 processor
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
# processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-large-xlsr-53")

input_values_list = list()
input_length_list = list()
labels_list = list()
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
remove_re = '[a-zA-Z0-9%]'


# text = special_filter(bracket_filter(text))
# if re.findall(remove_re, text) == []:
#     text_list.append(text)
#     audio_list.append(file)
#     durations += duration


def remove_special_characters(text: str) -> str:
    text = special_filter(bracket_filter(text))
    # text = re.sub(chars_to_ignore_regex, '', text)
    
    return text


def _read_txt_file(file_path:str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        
    return text

In [16]:
## kss 데이터셋
for folder in sorted(os.listdir(kss_path)):
    folder_path = os.path.join(kss_path, folder)
    for file in tqdm(sorted(os.listdir(folder_path))):
        if file.endswith(".wav"):
            wav_path = os.path.join(folder_path, file)
            # audio, _ = sf.read(wav_path)
            audio, _ = librosa.load(wav_path, sr=16000)  # 40분 걸림;;
            if audio.ndim > 1:                                     # array가 2차원일 경우 1차원으로 변환
                audio = np.delete(audio, 1, axis=1)
                audio = audio.reshape(-1)
            input_value = processor(audio, sampling_rate=16000).input_values[0]
            input_values_list.append(input_value)                  # processor를 거쳐 나온 오디오 array 값을 리스트에 저장
            txt_path = wav_path.replace(".wav", ".txt")
            text = remove_special_characters(_read_txt_file(txt_path))
            input_length_list.append(len(input_value))
            with processor.as_target_processor():
                labels_list.append(processor(text).input_ids)      # processor가 보유한 vocab에 맞는 인덱스 label값을 리스트에 저장

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11264/11264 [16:25<00:00, 11.43it/s]


In [8]:
## zeroth 데이터셋
for folder in tqdm(sorted(os.listdir(train_zeroth_path))):
    folder_path = os.path.join(train_zeroth_path, folder)
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".flac"):
            flac_path = os.path.join(folder_path, file)
            audio, _ = sf.read(flac_path)
            if audio.ndim > 1:
                audio = np.delete(audio, 1, axis=1)
                audio = audio.reshape(-1)
            
            input_value = processor(audio, sampling_rate=16000).input_values[0]
            input_values_list.append(input_value)                  # processor를 거쳐 나온 오디으 array 값을 리스트에 저장
            txt_path = flac_path.replace(".flac", ".txt")
            text = _read_txt_file(txt_path)
            input_length_list.append(len(input_value))
            with processor.as_target_processor():
                labels_list.append(processor(text).input_ids)      # proces

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [01:20<00:00,  1.30it/s]


In [ ]:
## ksponspeech 데이터셋 - 8~9분 걸림
for text, audio_path in tqdm(zip(text_list, audio_list)):
    audio, _ = sf.read(audio_path)
    non_silence_indices = del_noise(audio, top_db=30)  # del_noise 함수를 통해 노이즈 제거
    audio = np.concatenate([audio[start:end] for start, end in non_silence_indices])
    if audio.ndim > 1:
        audio = np.delete(audio, 1, axis=1)
        audio = audio.reshape(-1)
    input_value = processor(audio, sampling_rate=16000).input_values[0]
    input_values_list.append(input_value)
    input_length_list.append(len(input_value))
    text = remove_special_characters(text)
    with processor.as_target_processor():
        labels_list.append(processor(text).input_ids)  

In [ ]:
print(len(input_values_list), len(input_length_list), len(labels_list))

print(input_values_list[0], input_values_list[0].shape)
print(input_length_list[0])
print(labels_list[0])

In [ ]:
import pandas as pd

# df = pd.DataFrame({'array': wav_data_array, 'path': wav_path_list, 'text': txt_path_list})
train_rate = 0.90
train_idx = int(train_rate * len(input_values_list))

train_df = pd.DataFrame({'input_values': input_values_list[:train_idx], 'input_length': input_length_list[:train_idx], 'labels': labels_list[:train_idx]})
test_df = pd.DataFrame({'input_values': input_values_list[train_idx:], 'input_length': input_length_list[train_idx:], 'labels': labels_list[train_idx:]})

print(len(train_df))
print(len(test_df))

In [19]:
# 1차원을 넘는 데이터가 있는지 확인
for idx, values in enumerate(train_df['input_values']):
    if values.ndim > 1:
        print(idx)
        # array = np.delete(array, 1, axis=1)
        # print(array.shape)
        # array = array.reshape(-1)
        # print(array.shape)
        # df['array'][idx] = array
        
        
train_df.head()

,input_values,input_length,labels
0,"[-0.034159373, -0.051025007, -0.063998565, -0....",22016,"[518, 622, 870, 622, 777, 270]"
1,"[-0.111461304, -0.10138395, -0.039912038, 0.01...",73728,"[308, 1415, 622, 1367, 622, 467, 622, 561, 173..."
2,"[0.03649447, 0.03649447, 0.03882395, 0.0458123...",16384,"[1914, 183, 1900, 622, 1072, 1856, 391, 1162, ..."
3,"[-0.0014681732, 0.002090539, 0.0038698951, 0.0...",43520,"[1893, 301, 622, 1752, 622, 870, 694, 622, 545..."
4,"[0.009706883, 0.005026648, 0.010486921, 0.0097...",27136,"[1157, 270, 622, 1545, 1875, 622, 964, 1162, 6..."


In [ ]:
# 데이터프레임으로 구축된 학습 및 테스트용 데이터셋을 Datasets 모형으로 변환
# 
train_timit = Dataset.from_pandas(train_df)
test_timit = Dataset.from_pandas(test_df)

print(len(train_timit))
print(train_timit)

In [ ]:
# 훈련 데이터셋의 데이터 비중 확인
import matplotlib.pyplot as plt

cal_dataset = [int(length//16000) for length in train_timit['input_length']]

plt.title("dataset", fontsize=15)
plt.hist(cal_dataset, bins=10, alpha=.8, color='blue')
plt.ylabel("count", fontsize=13)
plt.xlabel("second", fontsize=13)
plt.grid()
plt.show()

In [ ]:
cal_dataset = [int(length//16000) for length in test_timit['input_length']]

plt.title("dataset", fontsize=15)
plt.hist(cal_dataset, bins=10, alpha=.8, color='blue')
plt.ylabel("count", fontsize=13)
plt.xlabel("second", fontsize=13)
plt.grid()
plt.show()

In [22]:
'''
긴 입력 시퀀스에는 많은 메모리가 필요하다. 
Wav2Vec2는 self-attention을 기반으로 하기 때문에 긴 입력 시퀀스에 대한 입력 길이에 따라 메모리 요구도 더욱 많아진다.
훈련 데이터셋에서 15초보다 긴 모든 시퀀스를 필터링한다.
'''
# max_input_length_in_sec = 15.0
max_input_length_in_sec = 6.0

train_timit = train_timit.filter(lambda x:x <= max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=['input_length'])
test_timit = test_timit.filter(lambda x:x <= max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=['input_length'])
train_timit  # 11568개에서 11030개로 축소

  0%|          | 0/212 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 211897
})

### 데이터확인

In [31]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_timit))

ipd.Audio(data=np.asarray(train_timit[rand_int]["input_values"]), rate=16000)  # 상대적으로 발성이 느리고 발음이 불문명
# ipd.Audio(data=np.asarray(train_timit[rand_int]["input_values"]), rate=44100)  # 원본과 똑같은 발성과 발음

In [21]:
import IPython.display as ipd
import numpy as np
import random

# rand_int = random.randint(0, len(train_timit))
rand_int = 0

# print(train_timit[rand_int]["text"])
# ipd.Audio(data=np.asarray(train_timit[rand_int]["input_values"]), rate=16000)  # 상대적으로 발성이 느리고 발음이 불문명
ipd.Audio(data=np.asarray(train_timit[rand_int]["input_values"]), rate=44100)  # 원본과 똑같은 발성과 발음
# ipd.Audio(data=np.asarray(train_dataset[0]["array"]), rate=44100)

In [29]:
import IPython.display as ipd
import numpy as np
import random

rand_int = 0

ipd.Audio(data=np.asarray(train_timit[rand_int]["input_values"]), rate=16000)  # 원본과 똑같은 발성과 발음

# 모델 학습
- 대부분의 NLP 모델과 달리 Wav2Vec2는 출력 길이보다 입력 길이가 훨씬 더 크다.
- 예를 들어, 입력 길이가 50000인 샘플의 출력 길이는 100을 넘지 않는다. 
- 큰 입력 크기가 주어지면 훈련 배치를 동적으로 채우는 것이 훨씬 더 효율적이다.
- 따라서 Wav2Vec2를 미세 조정하려면 아래에서 정의할 특별한 패딩 데이터 콜레이터(data_collator)가 필요하다.

### Trainer 구성

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
'''
음성에서 입력과 출력은 같은 padding 기능으로 처리되지 않는 형태이므로 data_collator가 필요하다.
일반적인 data_collator와 유사하도록 labels의 토큰을 -100으로 채움으로써 손실을 계산할 때 해당 토큰이 고려되지 않도록 한다.
'''
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
data_collator

In [ ]:
from datasets import load_metric

# evaluate metric을 정의
cer_metric = load_metric("cer")
# wer_metric = load_metric("wer")

In [ ]:
'''
-100을 pad_token_id로 바꾸고 id를 디코딩하여 인코딩된 레이블을 원래 문자열로 다시 변환한다.
그리고 동시에 연속적인 토큰이 CTC 스타일의 동일한 토큰으로 그룹화되지 않도록 한다.
'''
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    # wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}
    # return {"wer": wer}

# 모델 호출

In [ ]:
'''
사전 훈련된 Wav2Vec2 체크포인트 모델을 로드한다.
토크나이저의 pad_token_id는 모델의 pad_token_id를 정의하고 Wav2Vec2ForCTC의 경우 CTC의 blank_token도 정의해야 한다.
GPU 메모리를 절약하기 위해 PyTorch의 그래디언트 체크포인트(gradient checkpointing)를 활성화하고 손실 감소를 "mean"으로 설정한다.
'''

from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
# model = Wav2Vec2Model.from_pretrained(
    # "kresnik/wav2vec2-large-xlsr-korean",    # pre-trained된 한국어 wav2vec2 모델을 사용
    "facebook/wav2vec2-large-xlsr-53",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
)

print(model.config)
print("=" * 100)
print(model)

In [ ]:
'''
Wav2Vec2의 첫 번째 구성 요소는 최초 음성 신호에서 독립적인 특징을 추출하는 데 사용되는 CNN layer stack으로 구성되어있다.
모델의 이 부분은 사전 훈련(pre trained) 중에 이미 충분히 훈련되었으며 더 이상 미세 조정(fine tuning)할 필요가 없다.
따라서 feature extraction 부분의 모든 매개변수에 대해 requires_grad를 False로 설정할 수 있다. 즉, feature extraction에 대한 layer_freeze를 진행할 수 있다.
'''

model.freeze_feature_encoder()

# 학습 시작

In [ ]:
import wandb

wandb.login()
wandb.init(project='cis-awesome')

In [ ]:
from transformers import TrainingArguments

output_dir = os.path.join(dataset_path, "results")
# logs_dir = os.path.join(output_dir, "logs")
batch_size = 8

In [ ]:
training_args = TrainingArguments(
  output_dir=output_dir,           # 결과가 저징될 디렉토리 경로
  group_by_length=True,            # 입력 길이가 비슷한 훈련 데이터끼리 그룹화하여 휸련을 효율적으로 수행할 수 있으며 쓸모 없는 패딩 토큰의 전체 수를 크게 줄여 학습 시간을 단축한다.
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  evaluation_strategy="steps",
  # evaluation_strategy="epoch",
  num_train_epochs=100,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=1e-4,
  log_on_each_node=True,  # multi-gpu 사용시 trainer의 학습 log를 기록하는 인자(단일 gpu시 False 해도 무방)
  weight_decay=0.005,
  warmup_steps=1000,
  eval_accumulation_steps=1,
  # fsdp="full_shard",
  # logging_dir=logs_dir,  # "output_dir/runs/CURRENT_DATETIME_HOSTNAME*" 형식으로 저장되며 현재 output_dir 변수에 자동 적용된다.
  report_to="wandb",
  save_total_limit=200,
)

In [33]:
from transformers import Trainer
from setproctitle import setproctitle
setproctitle("Wav2Vec2 Train")

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    # train_dataset=timit["train"],
    # eval_dataset=timit["test"],
    train_dataset=train_timit,
    eval_dataset=test_timit,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
## kss_zeroth 훈련 시작
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.


In [25]:
# 훈련 결과
# trainer.state.log_history
for obj in trainer.state.log_history:
    print(obj)

{'loss': 48.8728, 'learning_rate': 1.9500000000000003e-05, 'epoch': 0.23, 'step': 200}
{'eval_loss': 90.04812622070312, 'eval_cer': 0.9996156740916297, 'eval_runtime': 173.6111, 'eval_samples_per_second': 6.417, 'eval_steps_per_second': 0.271, 'epoch': 0.23, 'step': 200}
{'loss': 22.6148, 'learning_rate': 3.9500000000000005e-05, 'epoch': 0.45, 'step': 400}
{'eval_loss': 11.44469928741455, 'eval_cer': 0.9999679728409692, 'eval_runtime': 170.2633, 'eval_samples_per_second': 6.543, 'eval_steps_per_second': 0.276, 'epoch': 0.45, 'step': 400}
{'loss': 5.6631, 'learning_rate': 5.95e-05, 'epoch': 0.68, 'step': 600}
{'eval_loss': 5.099677562713623, 'eval_cer': 0.9999679728409692, 'eval_runtime': 170.9855, 'eval_samples_per_second': 6.515, 'eval_steps_per_second': 0.275, 'epoch': 0.68, 'step': 600}
{'loss': 4.8401, 'learning_rate': 7.950000000000001e-05, 'epoch': 0.91, 'step': 800}
{'eval_loss': 5.076492786407471, 'eval_cer': 0.9999679728409692, 'eval_runtime': 171.5556, 'eval_samples_per_secon

In [26]:
# 훈련 저장
trainer.save_model(output_dir)

Saving model checkpoint to /wav2vec2/s-kr/fine-tune/dataset/results
Configuration saved in /wav2vec2/s-kr/fine-tune/dataset/results/config.json
Model weights saved in /wav2vec2/s-kr/fine-tune/dataset/results/pytorch_model.bin
Feature extractor saved in /wav2vec2/s-kr/fine-tune/dataset/results/preprocessor_config.json


# 추론

In [28]:
from setproctitle import setproctitle
setproctitle("wav2vec infer test")

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import os
import torch

dataset_path = "/wav2vec2/s-kr/fine-tune/dataset"
vocab_path = os.path.join(dataset_path, 'results/checkpoint-5700')

tokenizer = Wav2Vec2CTCTokenizer(vocab_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

IsADirectoryError: [Errno 21] Is a directory: '/wav2vec2/s-kr/fine-tune/dataset/results/checkpoint-5700'

In [29]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='', vocab_size=1261, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [30]:
model = Wav2Vec2ForCTC.from_pretrained(
    os.path.join(dataset_path, "results/checkpoint-5700")
).to('cuda')

# model.config

loading configuration file /wav2vec2/s-kr/fine-tune/dataset/results/checkpoint-5700/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer

In [31]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

In [32]:
results = test_timit.map(map_to_result, batch_size=2, remove_columns=test_timit.column_names)

  0%|          | 0/1286 [00:00<?, ?ex/s]

In [38]:
print(results['pred_str'][:5])
print("-" * 20)
print(results['text'][:5])

['이 표에있는 수자가 정화 한 건가요[UNK]', '저는 마음에 안데은 사람하고 있으면 금방 표가나요[UNK]', '적제한 표험니 안 더올르네요[UNK]', '그 사람은 진제한 표정으로 농당을해요[UNK]', '저는 우리나라 남장 표중 책격이에요[UNK]']
--------------------
['이 표에 있는 숫자가 정확한 건가요[UNK]', '저는 마음에 안 드는 사람하고 있으면 금방 표가 나요[UNK]', '적절한 표현이 안 떠오르네요[UNK]', '그 사람은 진지한 표정으로 농담을 해요[UNK]', '저는 우리나라 남자 표준 체격이에요[UNK]']


In [33]:
# 테스트셋 cer 확인
print("Test CER: {:.3f}".format(cer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test CER: 0.189


In [ ]:
# 데이터 추론 확인
model.to("cuda")

with torch.no_grad():
    logits = model(torch.tensor(test_timit[:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))